In [1]:
from datasets import load_dataset

/Users/dvanstrien/miniconda3/envs/biglam/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
SEARCH_TOKEN = "revolution"
SEARCH_TOKEN_CONTEXT_SIZE = 50

In [43]:
ds = load_dataset("davanstrien/hmd_newspapers")

Using custom data configuration davanstrien--hmd_newspapers-33689611c503c2bf
Reusing dataset parquet (/Users/dvanstrien/.cache/huggingface/datasets/davanstrien___parquet/davanstrien--hmd_newspapers-33689611c503c2bf/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8)
100%|██████████| 1/1 [00:00<00:00,  4.03it/s]


In [44]:
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'title', 'location', 'date', 'ocr_quality_mean'],
        num_rows: 2787068
    })
})

In [45]:
ds = ds["train"].filter(lambda x: SEARCH_TOKEN in x["text"].lower())

100%|██████████| 2788/2788 [03:00<00:00, 15.43ba/s]


In [46]:
ds

Dataset({
    features: ['text', 'title', 'location', 'date', 'ocr_quality_mean'],
    num_rows: 70403
})

In [47]:
def extract_token_context(search_token: str, text: str, contextsize: int = 50) -> str:
    text_tokens = text.split(" ")
    index = [i for i, token in enumerate(text_tokens) if search_token in token.lower()]
    # for i in index:
    #     after = text.split(" ")[i:i+contextsize]
    #     before = text.split(" ")[i-contextsize:i]
    #     yield " ".join(before+after)
    i = index[0]
    after = text.split(" ")[i : i + contextsize]
    before = text.split(" ")[i - contextsize : i]
    return " ".join(before + after)

In [48]:
def get_context(x, search_token=SEARCH_TOKEN, contextsize=SEARCH_TOKEN_CONTEXT_SIZE):
    text = x["text"]
    results = extract_token_context(search_token, text, contextsize=contextsize)
    # contexts = list(results)
    # num_exampes = len(contexts)
    # results = {k: [v]* num_exampes for k,v in x.items()}
    # results['context'] = contexts
    # return results
    return {"context": results}

In [49]:
ds = ds.map(
    get_context,
    fn_kwargs={"search_token": SEARCH_TOKEN, "contextsize": SEARCH_TOKEN_CONTEXT_SIZE},
    num_proc=8,
)

#0:   0%|          | 29/8801 [00:00<00:30, 288.72ex/s]









#0:   1%|          | 58/8801 [00:00<00:41, 211.16ex/s]














#0:   1%|          | 90/8801 [00:00<00:34, 251.52ex/s]




















#0:   1%|▏         | 130/8801 [00:00<00:28, 301.76ex/s]













#0:   2%|▏         | 170/8801 [00:00<00:25, 333.66ex/s]

















#0:   3%|▎         | 233/8801 [00:00<00:20, 428.17ex/s]
























#0:   3%|▎         | 278/8801 [00:00<00:24, 346.32ex/s]





















#0:   4%|▎         | 316/8801 [00:00<00:25, 332.08ex/s]



















#0:   4%|▍         | 352/8801 [00:01<00:28, 293.72ex/s]




















#0:   5%|▍         | 400/8801 [00:01<00:24, 337.61ex/s]




















#0:   5%|▍         | 438/8801 [00:01<00:24, 346.69ex/s]

























#0:   5%|▌         | 475/8801 [00:01<00:25, 325.82ex/s]

















#0:   6%|▌         | 515/8801 [00:01<00:24, 340.17ex/s]













#0:   6%|▋         | 551/8801 [00:01<00:24,

In [50]:
ds[10]

{'text': ',Tli mnas every daytrinother and him sOne day Sigreat quarre- black man to 11visaged butler anu(pent, but he kicks the c.the head of the other.call him step-moths "Cat,Methodisthe solem;father, whori 3 tenobjectionaiHe next runs-away\'from Ihim song hundreds of mites (rio slighthose days), and aftei• well; aiThejst.reconciliation (so like what foolish aille inraily d.) : nor even when-he gets him home, one(dludes to his -wayward conductever, comes over him for the better !Newcome never rabsence after his return, but was:quit6 gcwith him, and that night read the parrallu~ioSon iu-a-i\'cry low- and iiitiet voice. This hoWever Was oftemporary truce. War very soon broke out again bctwcthe impetuous lad and his rigid domineering motlier-in-laIt was not that he was very bad, or she perhaps more stcthan other ham\', but the two could not agree. • The bsulked and Was miserable at home.: Ile" fell to drink;with throoms in thStables„ .races, and was discovered ofIltfulliebrew cony(Mrs.

In [62]:
annotations_data = ds.remove_columns(["text"])

In [83]:
def clean_up(x):
    x["text"] = x.pop("context")
    x["meta"] = {k: v for k, v in x.items() if k != "text"}
    x = {k: x[k] for k in ("text", "meta")}
    return x

In [84]:
d = annotations_data.map(lambda x: {"text": x["context"], "meta": x.pop("context")})

100%|██████████| 70403/70403 [00:05<00:00, 12307.04ex/s]


In [92]:
d = annotations_data.map(
    clean_up,
    remove_columns=[
        "title",
        "location",
        "date",
        "ocr_quality_mean",
    ],
)

100%|██████████| 70403/70403 [00:06<00:00, 10650.99ex/s]


In [99]:
annotations_data_df = d.to_pandas()

In [100]:
annotations_data_df.rename({"context": "text"}, axis=1, inplace=True)

In [101]:
annotations_data_df

,text,meta
0,Cabinetrevolution at pon.stantinoblistate of a...,"{'date': 1853-09-26 00:00:00, 'location': 'Liv..."
1,the close oftheir deliberations.In succeeding ...,"{'date': 1853-09-27 00:00:00, 'location': 'Liv..."
2,risks ofdismemberment than at any time during ...,"{'date': 1853-09-27 00:00:00, 'location': 'Liv..."
3,"to him, witha look of scorn, ""do you think you...","{'date': 1853-09-27 00:00:00, 'location': 'Liv..."
4,with thehorrors of a European war gathering th...,"{'date': 1853-09-29 00:00:00, 'location': 'Liv..."
...,...,...
70398,"In 1823the Parliamentary Session, both in Fran...","{'date': 1824-02-09 00:00:00, 'location': 'Lon..."
70399,"authority.ofhis Majesty, has completely succee...","{'date': 1824-02-10 00:00:00, 'location': 'Lon..."
70400,which the people replied ; a battalion ofRoyal...,"{'date': 1824-02-13 00:00:00, 'location': 'Lon..."
70401,"9.We have this morning received, by Express, t...","{'date': 1824-02-16 00:00:00, 'location': 'Lon..."


In [102]:
annotations_data_df.to_json(
    "revolution_annotations.jsonl", orient="records", lines=True
)